In [1]:
import pandas as pd
import numpy as np
import cchardet
import re
from textrank4zh import TextRank4Keyword, TextRank4Sentence
import jieba
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings("ignore")

In [42]:
class textSummary(object):
    def __init__(self,N):
        super().__init__()
        self.N = N
    
    def drop_brackets_content(self,sentence):
        """
        清除括号中内容
        """
        pattern  = "（[\s\S]*?）"
        res = re.sub(pattern,"",sentence)
        return res
    
    def clean_sentence(self,sentence):
        """
        去掉空格和换行符
        """
        clean_sentence = sentence.replace("\r\n","").replace("　　","")
        return clean_sentence
    
    def save_quotation_marks_content_to_dict(self,sentence):
        """
        引号中话可能带着分隔符，断句的话会将原本一句话的内容，分割成多句，所以保留引号中的content，让其免于分割；
        将其保存在一个字典中，用于后续还原
        """
        pattern  = "“[\s\S]*?”"
        res = re.findall(pattern,sentence)
        save_quote_content_dict = {}
        for index,_r in enumerate(res):
            new_str = "waws520_" + str(index)
            save_quote_content_dict[new_str] = _r
            sentence = sentence.replace(_r,new_str,1)
        return save_quote_content_dict,sentence
    
    def replace_quotation_marks_dict_back_origin_sentence(self,save_quote_content_dict,sentence):
        """
        对之前的存在替换字典中的内容进行还原
        """
        for key,value in save_quote_content_dict.items():
            if key in sentence:
                sentence = sentence.replace(key,value,1)
        return sentence
    
    def cut_sents(self,content):
        """
        对句子完成切割--分句
        """
        # 在分割前先进行引号内容处理
        save_quote_content_dict,sentence = self.save_quotation_marks_content_to_dict(content)
        sentences = re.split(r"([。!！?？；;])", sentence)
        sentences.append("")
        sentences = ["".join(i) for i in zip(sentences[0::2],sentences[1::2])]
        new_sentences = []
        for index,sentence in enumerate(sentences):
            if sentence == "":
                continue
            new_sentences.append(self.replace_quotation_marks_dict_back_origin_sentence(save_quote_content_dict,sentence))
        print("------ 分句后的结果：开始-------")
        for index,sentence in enumerate(new_sentences):
            print("第{}句：{}".format(index,sentence))
        print("------ 分句后的结果：结束-------")
        return new_sentences
    
    def clean_and_drop_brackets_content(self,sentence):
        """
        清除括号内容和空格
        """
        return self.clean_sentence(self.drop_brackets_content(sentence))
    
    def segmentation_sentence(self,sentence):
        """
        清理加分句
        """
        return self.cut_sents(self.clean_sentence(self.drop_brackets_content(sentence)))
    
    
    def save_key_word(self,sentence):
        """
        通过textrank获取关键词
        """
        tr4w = TextRank4Keyword()
        tr4w.analyze(text=sentence, lower=True, window=2)
        save_key_word_list = []
        for item in tr4w.get_keywords(50, word_min_len=1):
            save_key_word_list.append((item.word, item.weight))
        return save_key_word_list
    
    
    def cut_word_test(self,context_list):
        """
        切词获取所有单词的列表，制作词向量的前序操作
        """
        stopkey=[line.strip() for line in open('stopwords.txt',encoding='utf-8').readlines()]
        total_cutword = []
        for i in context_list:
            words=jieba.cut(i)
            words_filter=[word for word in words if word not in stopkey]
            if len(words_filter) !=0:
                total_cutword.append(words_filter)
        return total_cutword
    
    def get_w2v_model(self,data,flag=1):
        """
        拿到word2vec模型，flag=0,重新训练模型；flag=1,直接调用已经存在的模型
        """
        if flag == 0:
            print("计算词向量中，请稍后...")
            model = Word2Vec(data, size=256, window=5,iter=1000, min_count=1, workers=4)
            model.save('model.bin')
        else:
            print("正在加载词向量")
            model = Word2Vec.load('model.bin')
        return model
    
    def get_similar_word(self,key_word_list,model):
        """
        通过余弦相似度，完成对每个关键词的相似词的搜寻
        """
        print("正在进行关键词相似词的搜寻")
        new_key_word = []
        new_word_list = []
        for key,value in tqdm(key_word_list,total=len(key_word_list)):
            new_key_word.append((key,value))
            new_word_list.append(key)
            if key in model.wv.index2word:
                _index = model.wv.index2word.index(key)
                for index,word in enumerate(model.wv.index2word):
                    if index == _index:
                        continue
                    else:
                        cosine_value = cosine_similarity([model.wv.vectors[index],model.wv.vectors[_index]])
                        if cosine_value[0][1] > 0.8:
                            # print("{}和{}是相似的，相似度是：{}".format(word,key,cosine_value[0][1]))
                            if word not in new_word_list:
                                new_key_word.append((word,value))
                                new_word_list.append(word)
        return new_key_word
    
    def get_sentence_score_list(self,sentence,title,new_key_word):
        """
        得到句子的重要程度分数，默认title的分数是0.5，其他的句子的分数是句子中关键词的分数之和，最后按照分数由高到低排序
        """
        print("正在获取句子的重要程度分数")
        new_sentence = self.segmentation_sentence(sentence)
        sentence_score_list = []
        for index,_sentence in enumerate(new_sentence):
            score = 0
            for key,value in new_key_word:
                if key in _sentence:
                    score += value
            sentence_score_list.append((_sentence,score))
        if title[-1] not in ["。","!","！","?","？","；",";"]:
            title = title + "。"
        sentence_score_list.append((title,0.5))
        sentence_score_list.sort(key=lambda x:-x[1])
        return sentence_score_list
    
    def output_summary(self,sentence_score_list):
        """
        按照句子的分数取出句子进行拼接，直到逼近字数N为止
        """
        sentence = ""
        for key,value in sentence_score_list:
            if len(sentence) < self.N:
                sentence += key
            else:
                break
        return sentence

In [43]:
def get_input_artical_summary(article="",title="",_index= -1,flag = 1):
    """
    第一种：_index为-1，需要输入article和title的值，进行自定义文章摘要提取；
    第二种：_index不为-1，指定chinese_news的文章id进行摘要提取
    flag 如果有Word2vec的model,选1，没有模型，或者想重新训练，选0；
    """
    df = pd.read_csv("chinese_news.csv",encoding="gb18030")
    df = df.dropna(subset = ['content',"title"])
    df = df.reset_index(drop=True)
    
    # 当前默认摘要字数：100
    ts = textSummary(100)
    
    # 截断数据，小规模测试
    df = df[:201]
    df.insert(loc=2, column='key_word', value=0)
    if _index == -1:
        if article != "" and title != "":
            print("当前填入自定义文章，进行摘要提取")
            _index = df.shape[0]
            df.loc[_index] = [0,0,0,0,article,0,title,0]
        else:
            print("当前参数有误：第一种：_index为-1，需要输入article和title的值，进行自定义文章摘要提取；第二种：_index不为-1，指定chinese_news的文章id进行摘要提取")
    else:
        print("当前_index的值不为-1，进行chinese_news指定文章的摘要提取")

    # 看不见执行进度，不用了
    # new_df["key_word"] = new_df["content"].apply(save_key_word) 

    print("当前正在获取关键词，请稍等...")
    for index,content in tqdm(enumerate(df["content"]),total=len(list(df["content"]))):
        df["key_word"][index] = ts.save_key_word(content) 
    
    total_cutword = ts.cut_word_test([ts.clean_and_drop_brackets_content(i) for i in list(df["content"])])
    model = ts.get_w2v_model(total_cutword,flag=flag)
    new_key_word = ts.get_similar_word(df["key_word"][_index],model)
    return ts.output_summary(ts.get_sentence_score_list(df["content"][_index],df["title"][_index],new_key_word))

In [40]:
# 自定义文章和title,进行摘要提取
get_input_artical_summary("从赏月地图来看，今年中秋节当天（9月10日），西北地区和东南沿海赏月天气条件较好，新疆、青海、甘肃、宁夏、陕西、内蒙古中西部及苏皖北部、浙江南部、福建、广东东部等地都将是皓月当空，能够清晰地看到天空中“白玉盘”般的圆月。东北大部、京津冀、山东、河南中北部、苏皖南部、湖北、重庆、广东西部、广西等地都将是多云或阴天，上演彩云追月，圆月时而冒头时而隐身，在云朵映衬下增添一份神秘感。而在山西大部、河北西部以及贵州、云南、四川盆地南部、西藏大部等地，云量较多，一些地方还会下雨，或许圆月难见，想一睹月亮真容，可以通过直播来云赏月。虽然是共赏同一轮明月，但赏月方式却是五花八门，既有充满诗情画意的登高望月、泛舟赏月等传统方式，随着科技的进步，如今还有合拍赏月、摩天轮赏月等新型赏月方式。在北方赏月，登高望月、合拍赏月等方式皆可解锁。在敦煌月牙泉，因当地纬度较高，还能享受“手可摘月亮”的极致体验，通过借位可拍出将月亮“托起”的照片，分分钟让你成为朋友圈最靓的仔。长春净月潭则可欣赏到“明月松间照”的美景，在松林的衬托下，月色更显静谧。南方水系众多，自古以来人们的赏月活动多与水有关。在杭州，西湖胜景平湖秋月、三潭印月是人们熟知的赏月胜地，泛舟湖上，能体会到“烟笼秋水月笼纱”的诗般意境。而黄山风景区拥有多个赏月佳地，光明顶最为开阔，登高望圆月一览无遗。如今合拍赏月和摩天轮赏月也越来越受人追捧。上海外滩及广州小蛮腰都是合拍赏月的胜地。南昌赣江沿岸的“南昌之星”摩天轮，也可让月亮如在眼前，唾手可得。","十五的月亮十五圆 今年中秋哪里可见皓月当空？")

当前填入自定义文章，进行摘要提取
当前正在获取关键词，请稍等...


  0%|          | 0/202 [00:00<?, ?it/s]

正在加载词向量
正在进行关键词相似词的搜寻


  0%|          | 0/50 [00:00<?, ?it/s]

正在获取句子的重要程度分数
------ 分句后的结果：开始-------
第0句：从赏月地图来看，今年中秋节当天，西北地区和东南沿海赏月天气条件较好，新疆、青海、甘肃、宁夏、陕西、内蒙古中西部及苏皖北部、浙江南部、福建、广东东部等地都将是皓月当空，能够清晰地看到天空中“白玉盘”般的圆月。
第1句：东北大部、京津冀、山东、河南中北部、苏皖南部、湖北、重庆、广东西部、广西等地都将是多云或阴天，上演彩云追月，圆月时而冒头时而隐身，在云朵映衬下增添一份神秘感。
第2句：而在山西大部、河北西部以及贵州、云南、四川盆地南部、西藏大部等地，云量较多，一些地方还会下雨，或许圆月难见，想一睹月亮真容，可以通过直播来云赏月。
第3句：虽然是共赏同一轮明月，但赏月方式却是五花八门，既有充满诗情画意的登高望月、泛舟赏月等传统方式，随着科技的进步，如今还有合拍赏月、摩天轮赏月等新型赏月方式。
第4句：在北方赏月，登高望月、合拍赏月等方式皆可解锁。
第5句：在敦煌月牙泉，因当地纬度较高，还能享受“手可摘月亮”的极致体验，通过借位可拍出将月亮“托起”的照片，分分钟让你成为朋友圈最靓的仔。
第6句：长春净月潭则可欣赏到“明月松间照”的美景，在松林的衬托下，月色更显静谧。
第7句：南方水系众多，自古以来人们的赏月活动多与水有关。
第8句：在杭州，西湖胜景平湖秋月、三潭印月是人们熟知的赏月胜地，泛舟湖上，能体会到“烟笼秋水月笼纱”的诗般意境。
第9句：而黄山风景区拥有多个赏月佳地，光明顶最为开阔，登高望圆月一览无遗。
第10句：如今合拍赏月和摩天轮赏月也越来越受人追捧。
第11句：上海外滩及广州小蛮腰都是合拍赏月的胜地。
第12句：南昌赣江沿岸的“南昌之星”摩天轮，也可让月亮如在眼前，唾手可得。
------ 分句后的结果：结束-------


'十五的月亮十五圆 今年中秋哪里可见皓月当空？从赏月地图来看，今年中秋节当天，西北地区和东南沿海赏月天气条件较好，新疆、青海、甘肃、宁夏、陕西、内蒙古中西部及苏皖北部、浙江南部、福建、广东东部等地都将是皓月当空，能够清晰地看到天空中“白玉盘”般的圆月。'

In [41]:
# 输入0~200之间的标号，对chinese_news指定的文章进行摘要提取
# 下面输出的第11句，解决了引号中存在分隔符导致分句不对的情况
get_input_artical_summary(_index=6)

当前_index的值不为-1，进行chinese_news指定文章的摘要提取
当前正在获取关键词，请稍等...


  0%|          | 0/201 [00:00<?, ?it/s]

正在加载词向量
正在进行关键词相似词的搜寻


  0%|          | 0/50 [00:00<?, ?it/s]

正在获取句子的重要程度分数
------ 分句后的结果：开始-------
第0句：虽然至今夏普智能手机在市场上无法排得上号，已经完全没落，并于 2013 年退出中国市场，但是今年 3 月份官方突然宣布回归中国，预示着很快就有夏普新机在中国登场了。
第1句：那么，第一款夏普手机什么时候登陆中国呢？
第2句：又会是怎么样的手机呢？
第3句：近日，一款型号为 FS8016 的夏普神秘新机悄然出现在 GeekBench 的跑分库上。
第4句：从其中相关信息了解到，这款机子并非旗舰定位，所搭载的是高通骁龙 660 处理器，配备有 4GB 的内存。
第5句：骁龙 660 是高通今年最受瞩目的芯片之一，采用 14 纳米工艺，八个 Kryo 260 核心设计，集成 Adreno 512 GPU 和 X12 LTE 调制解调器。
第6句：当前市面上只有一款机子采用了骁龙 660 处理器，那就是已经上市销售的 OPPO R11。
第7句：骁龙 660 尽管并非旗舰芯片，但在多核新能上比去年骁龙 820 强，单核改进也很明显，所以放在今年仍可以让很多手机变成高端机。
第8句：不过，由于 OPPO 与高通签署了排他性协议，可以独占两三个月时间。
第9句：考虑到夏普既然开始测试新机了，说明只要等独占时期一过，夏普就能发布骁龙 660 新品了。
第10句：按照之前被曝光的渲染图了解，夏普的新机核心竞争优势还是全面屏，因为从 2013 年推出全球首款全面屏手机 EDGEST 302SH 至今，夏普手机推出了多达 28 款的全面屏手机。
第11句：在 5 月份的媒体沟通会上，惠普罗忠生表示：“我敢打赌，12 个月之后，在座的各位手机都会换掉。因为全面屏时代的到来，我们怀揣的手机都将成为传统手机。”
------ 分句后的结果：结束-------


'配骁龙660 全面屏鼻祖夏普新机酝酿中。虽然至今夏普智能手机在市场上无法排得上号，已经完全没落，并于 2013 年退出中国市场，但是今年 3 月份官方突然宣布回归中国，预示着很快就有夏普新机在中国登场了。'

### DONE
- [x] 使用textrank获取关键词
- [x] 使用word2vec词向量的训练
- [x] 相似词语的计算
- [x] 文本摘要的简易版本
- [x] 对于引号中的存在分隔符的文本，单独处理

### TODO
- [ ] 语义通顺，暂时没有方案